# Example EEG Feature Extraction

In [ ]:
!pip install mne
!pip install numpy
!pip install pickle
!pip install mne_features

In [ ]:
import mne as mne
import numpy as np
import pickle
import pandas as pd
from mne_features.feature_extraction import extract_features, FeatureExtractor

In [ ]:
def load_object(fname):
    try:
        with open(fname + ".pickle", "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)
        
eeg_ec_freq_bands = load_object(r"data/example_eeg/example_subj_EC_preproc")
eeg_eo_freq_bands = load_object(r"data/example_eeg/example_subj_EO_preproc")

In [ ]:
def make_epochs(raw, dur, ovl):
    epochs = mne.make_fixed_length_epochs(
        raw,
        duration=dur,
        overlap=ovl,
        preload=True,
        verbose=False
    )
    return np.array(epochs.get_data(units='uV'))

In [ ]:
feat_extc = {
    'only_whole_spec': {
        'pow_freq_bands',
        'wavelet_coef_energy',
    },
    'methods': {
        'std',
        'spect_slope',
        'svd_fisher_info',
        'hjorth_complexity',
        'hjorth_complexity_spect',
        'ptp_amp',
        'quantile',
        'line_length',
        'zero_crossings',
        'skewness',
        'kurtosis',
        'higuchi_fd',
        'samp_entropy',
        'app_entropy',
        'spect_entropy',
        'mean',
        'hurst_exp'
    }
}

freq_bands_of_interest = {
    'delta': [0.5, 4],
    'theta': [4, 8],
    'alpha': [8, 14],
    'beta': [14, 30],
    'whole_spec': [0.5, 30],
}

In [ ]:
def get_param_for_methods(methods, fboi, fb):
    param = dict()
    if 'quantile' in methods:
        param['quantile__q'] = np.array([0.05, 0.25, 0.75, 0.95])
    if 'pow_freq_bands' in methods:
        param['pow_freq_bands__freq_bands'] = fboi
        param['pow_freq_bands__log'] = True
        param['pow_freq_bands__ratios_triu'] = False
        param['pow_freq_bands__ratios'] = None
    if 'energy_freq_bands' in methods:
        param['energy_freq_bands__freq_bands'] = fboi
    if 'spect_slope' in methods:
        param['spect_slope__fmin'] = np.array([fboi[fb][0]])
        param['spect_slope__fmax'] = np.array([fboi[fb][1]])
        param['spect_slope__with_intercept'] = True
    return param

def extract_methods(eeg_fb_dict, epoch_dur, epoch_ovl, set_methods=feat_extc, fboi=freq_bands_of_interest):
    methods_whole_spec = set_methods['only_whole_spec']
    methods = set_methods['methods']
    
    fb_epochs = dict()
    for key in eeg_fb_dict.keys():
        print(f"Extracting features in frequency band {key}")
        raw_fb = eeg_fb_dict[key]
        
        epochs = make_epochs(raw_fb.copy(), epoch_dur, epoch_ovl)
        if key == 'whole_spec':
            methods = set(methods).union(methods_whole_spec)
        
        param = get_param_for_methods(methods, fboi, key)
        
        features = extract_features(
            X=epochs,
            sfreq=raw_fb.info['sfreq'],
            selected_funcs=methods,
            funcs_params=param,
            n_jobs=5,
            ch_names=raw_fb.info['ch_names'],
            return_as_df=True
        )
        fb_epochs[key] = features
    return fb_epochs

In [ ]:
print("Extracting features from eyes closed eeg")
eeg_ec_features = extract_methods(
    eeg_ec_freq_bands, 
    epoch_dur=3.9, epoch_ovl=0, 
    set_methods=feat_extc, 
    fboi=freq_bands_of_interest
)
print("")
print("Extracting features from eyes opened eeg")
eeg_eo_features = extract_methods(
    eeg_eo_freq_bands, 
    epoch_dur=1.9, epoch_ovl=0, 
    set_methods=feat_extc, 
    fboi=freq_bands_of_interest
)

In [ ]:
print(eeg_ec_features['delta']['mean'])